# Install git-lfs and Download the Model

In [ ]:
%%sh

conda install -c conda-forge git-lfs
git lfs install
git lfs fetch --all
git lfs pull
git lfs checkout

# Package the Model

In [ ]:
import os
import tarfile

model_path = "model/"
code_path = "code/"

zipped_model_path = "model.tar.gz"

with tarfile.open(zipped_model_path, "w:gz") as tar:
    tar.add("pytorch_model.bin", os.path.join(model_path, "pytorch_model.bin"))
    tar.add("sagemaker_infer.py", os.path.join(code_path, "infer.py"))
    tar.add("model.py", os.path.join(code_path, "model.py"))
    tar.add("sagemaker_requirements.txt", os.path.join(code_path, "requirements.txt"))

# Deploy the Model

In [ ]:
import time

from sagemaker import get_execution_role, Session
from sagemaker.pytorch import PyTorchModel
from sagemaker.serverless import ServerlessInferenceConfig

algorithm_name = "bert-small-ko-word-boundary-detection"
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
endpoint_name = f"{algorithm_name}-{timestamp}"

model = PyTorchModel(
    name=f"{algorithm_name}-model-{timestamp}",
    entry_point="infer.py",
    model_data=zipped_model_path,
    role=get_execution_role(),
    framework_version="2.0",
    py_version="py310",
)

config = ServerlessInferenceConfig(
    memory_size_in_mb=3072,
    max_concurrency=4,
)

predictor = model.deploy(
    serverless_inference_config=config,
    endpoint_name=endpoint_name
)

[12/02/24 15:56:24] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=490559;file:///Users/reyaes/Repos/bert-small-ko-word-boundary-detection/env/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=638679;file:///Users/reyaes/Repos/bert-small-ko-word-boundary-detection/env/lib/python3.12/site-packages/botocore/credentials.py#1278\1278]8;;\

[12/02/24 15:56:25] WARNING  Couldn't call 'get_role' to get Role ARN from role name jstover to get ]8;id=973155;file:///Users/reyaes/Repos/bert-small-ko-word-boundary-detection/env/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=205821;file:///Users/reyaes/Repos/bert-small-ko-word-boundary-detection/env/lib/python3.12/site-packages/sagemaker/session.py#5902\5902]8;;\
                             Role path.                                                                            

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11 model = PyTorchModel(                                                                       │
│   12 │   entry_point="infer.py",                                                                 │
│   13 │   model_data=zipped_model_path,                                                           │
│ ❱ 14 │   role=get_execution_role(),                                                              │
│   15 │   framework_version="2.0",                                                                │
│   16 │   py_version="py310",                                                                     │
│   17 )                                                                                           │
│                                                                                                  │
│ /Users/reyaes/Repos/bert-small-ko-word-boundary-detection/env/lib/python3.12/site-packages/sagem │
│ aker/session.py:7864 in get_execution_role                                                       │
│                                                                                                  │
│   7861 │   │   "The current AWS identity is not a role: {}, therefore it cannot be used as a "   │
│   7862 │   │   "SageMaker execution role"                                                        │
│   7863 │   )                                                                                     │
│ ❱ 7864 │   raise ValueError(message.format(arn))                                                 │
│   7865                                                                                           │
│   7866                                                                                           │
│   7867 def generate_default_sagemaker_bucket_name(boto_session):                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: The current AWS identity is not a role: arn:aws:iam::732878590761:user/jstover, therefore it cannot be 
used as a SageMaker execution role

# Test the Endpoint

In [ ]:
import json

client = Session().sagemaker_runtime_client
body = ["공장 훗날"]

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(body, ensure_ascii=False),
    ContentType="application/json",
)

response["Body"].read()